<a href="https://colab.research.google.com/github/yoojungsun0/AdvImgSeg/blob/master/Linear.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install thop

In [2]:


import argparse

import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from thop import profile, clever_format
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR10
from tqdm import tqdm


from google.colab import drive
drive.mount('/content/gdrive/')

import sys
%cd /content/gdrive/MyDrive/'Colab Notebooks'/SimCLR


import utils
from model_separation import Model

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).
/content/gdrive/MyDrive/Colab Notebooks/SimCLR


In [3]:
# class Net(nn.Module):
#     def __init__(self, num_class, pretrained_path):
#         super(Net, self).__init__()

#         # encoder
#         self.f = Model().f
#         # classifier
#         self.fc = nn.Linear(2048, num_class, bias=True)
#         self.load_state_dict(torch.load(pretrained_path, map_location='cpu'), strict=False)

#     def forward(self, x):
#         x = self.f(x)
#         feature = torch.flatten(x, start_dim=1)
#         out = self.fc(feature)
#         return out
class Net(nn.Module):
    def __init__(self, num_class, pretrained_path):
        super(Net, self).__init__()

        # encoder
        self.f = Model().f
        # classifier
        self.fc = nn.Linear(2048, num_class, bias=True)
        self.load_state_dict(torch.load(pretrained_path, map_location='cpu'), strict=False)

    def forward(self, x):
        x = self.f(x)
        x1 = x[:,:,0,:]
        x2 = x[:,:,1,:]

        feature1 = torch.flatten(x1, start_dim=1)
        feature2 = torch.flatten(x2, start_dim=1)
        # out1 = self.g1(feature1)
        # out2 = self.g2(feature2)
        # feature = torch.flatten(x, start_dim=1)
        out = self.fc(feature1)
        return out

# train or test for one epoch
def train_val(net, data_loader, train_optimizer):
    is_train = train_optimizer is not None
    net.train() if is_train else net.eval()

    total_loss, total_correct_1, total_correct_5, total_num, data_bar = 0.0, 0.0, 0.0, 0, tqdm(data_loader)
    with (torch.enable_grad() if is_train else torch.no_grad()):
        for data, target in data_bar:
            data, target = data.cuda(non_blocking=True), target.cuda(non_blocking=True)
            out = net(data)
            loss = loss_criterion(out, target)

            if is_train:
                train_optimizer.zero_grad()
                loss.backward()
                train_optimizer.step()

            total_num += data.size(0)
            total_loss += loss.item() * data.size(0)
            prediction = torch.argsort(out, dim=-1, descending=True)
            total_correct_1 += torch.sum((prediction[:, 0:1] == target.unsqueeze(dim=-1)).any(dim=-1).float()).item()
            total_correct_5 += torch.sum((prediction[:, 0:5] == target.unsqueeze(dim=-1)).any(dim=-1).float()).item()

            data_bar.set_description('{} Epoch: [{}/{}] Loss: {:.4f} ACC@1: {:.2f}% ACC@5: {:.2f}%'
                                     .format('Train' if is_train else 'Test', epoch, epochs, total_loss / total_num,
                                             total_correct_1 / total_num * 100, total_correct_5 / total_num * 100))

    return total_loss / total_num, total_correct_1 / total_num * 100, total_correct_5 / total_num * 100

In [4]:
# import os
# rootdir = '../'
model_path = 'results/Model3_twovec/128_0.5_200_256_100_model.pth'
# model_path = os.path.join(rootdir, path)
#'results/128_0.5_200_256_100_model.pth'
batch_size = 256 #512
epochs = 50 #100

train_data = CIFAR10(root='data', train=True, transform=utils.train_transform, download=True)
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=16, pin_memory=True)
test_data = CIFAR10(root='data', train=False, transform=utils.test_transform, download=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=16, pin_memory=True)

model = Net(num_class=len(train_data.classes), pretrained_path=model_path).cuda()
for param in model.f.parameters():
    param.requires_grad = False

flops, params = profile(model, inputs=(torch.randn(1, 3, 32, 32).cuda(),))
flops, params = clever_format([flops, params])
print('# Model Params: {} FLOPs: {}'.format(params, flops))
optimizer = optim.Adam(model.fc.parameters(), lr=1e-3, weight_decay=1e-6)
loss_criterion = nn.CrossEntropyLoss()
results = {'train_loss': [], 'train_acc@1': [], 'train_acc@5': [],
            'test_loss': [], 'test_acc@1': [], 'test_acc@5': []}

best_acc = 0.0
for epoch in range(1, epochs + 1):
    train_loss, train_acc_1, train_acc_5 = train_val(model, train_loader, optimizer)
    results['train_loss'].append(train_loss)
    results['train_acc@1'].append(train_acc_1)
    results['train_acc@5'].append(train_acc_5)
    test_loss, test_acc_1, test_acc_5 = train_val(model, test_loader, None)
    results['test_loss'].append(test_loss)
    results['test_acc@1'].append(test_acc_1)
    results['test_acc@5'].append(test_acc_5)
    # save statistics
    data_frame = pd.DataFrame(data=results, index=range(1, epoch + 1))
    data_frame.to_csv('results/Model3_twovec/linear_statistics.csv', index_label='epoch')
    if test_acc_1 > best_acc:
        best_acc = test_acc_1
        torch.save(model.state_dict(), 'results/Model3_twovec/linear_model.pth')



Files already downloaded and verified
Files already downloaded and verified


  0%|          | 0/196 [00:00<?, ?it/s]

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_bn() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[WARN] Cannot find rule for <class 'torch.nn.modules.container.Sequential'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'torchvision.models.resnet.Bottleneck'>. Treat it as zero Macs and zero Params.
[INFO] Register count_adap_avgpool() for <class 'torch.nn.modules.pooling.AdaptiveAvgPool2d'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[WARN] Cannot find rule for <class '__main__.Net'>. Treat it as zero Macs and zero Params.
# Model Params: 23.52M FLOPs: 1.30G


Train Epoch: [1/50] Loss: 0.8557 ACC@1: 70.83% ACC@5: 97.05%: 100%|██████████| 196/196 [00:50<00:00,  3.90it/s]
Test Epoch: [1/50] Loss: 0.6072 ACC@1: 78.63% ACC@5: 98.97%: 100%|██████████| 40/40 [00:05<00:00,  7.37it/s]
Train Epoch: [2/50] Loss: 0.7648 ACC@1: 73.11% ACC@5: 97.80%: 100%|██████████| 196/196 [00:50<00:00,  3.90it/s]
Test Epoch: [2/50] Loss: 0.5848 ACC@1: 79.44% ACC@5: 99.00%: 100%|██████████| 40/40 [00:05<00:00,  7.52it/s]
Train Epoch: [3/50] Loss: 0.7498 ACC@1: 73.82% ACC@5: 97.80%: 100%|██████████| 196/196 [00:50<00:00,  3.87it/s]
Test Epoch: [3/50] Loss: 0.5517 ACC@1: 81.06% ACC@5: 99.11%: 100%|██████████| 40/40 [00:05<00:00,  7.59it/s]
Train Epoch: [4/50] Loss: 0.7473 ACC@1: 73.89% ACC@5: 97.82%: 100%|██████████| 196/196 [00:50<00:00,  3.88it/s]
Test Epoch: [4/50] Loss: 0.5676 ACC@1: 80.09% ACC@5: 99.23%: 100%|██████████| 40/40 [00:05<00:00,  7.47it/s]
Train Epoch: [5/50] Loss: 0.7332 ACC@1: 74.28% ACC@5: 97.87%: 100%|██████████| 196/196 [00:51<00:00,  3.81it/s]
Test